# Concatenate garbage data from Arroyo Vivo

This notebook takes as input 21 csvs from Arroyo Vivo and creates one csv deleting duplicates.

Most csvs have different column names for garbage categories. The output orders data the following way:

* "itemname" --> How the item was used (Bolsa de Plastico para comestibles, Vasos o tapas de unicel o Plastico)
* "tipo" --> middle category
* "material" --> Material of item (Vidrio, Plastico, Papel, Unicel etc)

#Original column names:

* "itemname" <--"itemname", "T"
* "tipo" <-- "material"
* "material" <-- "id", "category", "Category", "Column1", "tipo"

Notes:
* csv "2022_2023" underwent an external coordenate-fix-process. It's data treatment is slightly different.
* csv "mdt-data abril a junio 2023.csv" and csv "'mdt-data agosto 2023.csv" didn't have a column for "material", it was created from data in "tipo".
* Materials 'Colector de aguas pluviales','Comercio','Recoleccion','Recipiente de basura', 'Reciclaje', 'Canal', 'Colector','Basurero', 'Hidrología' were deleted before generating final output.

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [154]:
res = 10
# Referencia
schema = 'projects_research'
table = 'arroyovivo_residuos_point'
query = f"SELECT * FROM {schema}.{table}"
residuos_point = aup.gdf_from_query(query, geometry_col='geometry')

In [155]:
# Show
print(residuos_point.shape)
residuos_point.head(1)

(16179, 9)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin,geometry
0,Latas de aluminio o estaño,None,Aluminio,11,25.644154,-100.310618,627.104902,9 de enero.csv,MULTIPOINT (-100.31062 25.64415)


In [220]:
calculo = residuos_point.loc[residuos_point.itemname == 'Ropa o zapatos']
calculo.material.unique()

array(['Tela'], dtype=object)

In [177]:
ejemplo = calculo.loc[calculo.material=='Vidrio']
ejemplo.shape

(222, 9)

## Import and treatment of base data

In [5]:
tmp_dir = f'../../data/external/temporal_todocker/arroyo_vivo/input_csv_2/'

### 29_0ctubre_mdt-data.csv

In [12]:
doc_29_0ctubre_mdt_data = pd.read_csv(tmp_dir+'29_0ctubre_mdt-data.csv',index_col=False)

/tmp/ipykernel_20994/480549487.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_29_0ctubre_mdt_data = pd.read_csv(tmp_dir+'29_0ctubre_mdt-data.csv',index_col=False)


In [17]:
doc_29_0ctubre_mdt_data.columns

Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

### mdt-data (1).csv

In [21]:
doc_mdt_data_1 = pd.read_csv(tmp_dir+'mdt-data (1).csv',index_col=False)

/tmp/ipykernel_20994/2574467936.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_1 = pd.read_csv(tmp_dir+'mdt-data (1).csv',index_col=False)


In [23]:
doc_mdt_data_1.columns

Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

### mdt-data 5julio-24sep.csv

In [24]:
doc_mdt_data_5julio_24sep = pd.read_csv(tmp_dir+'mdt-data 5julio-24sep.csv',index_col=False)

/tmp/ipykernel_20994/225545345.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_5julio_24sep = pd.read_csv(tmp_dir+'mdt-data 5julio-24sep.csv',index_col=False)


In [26]:
doc_mdt_data_5julio_24sep.columns

Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

### mdt-data 22ago-24sep.csv

In [28]:
doc_mdt_data_22ago_24sep = pd.read_csv(tmp_dir+'mdt-data 22ago-24sep.csv',index_col=False)

/tmp/ipykernel_20994/893118776.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_22ago_24sep = pd.read_csv(tmp_dir+'mdt-data 22ago-24sep.csv',index_col=False)


In [30]:
doc_mdt_data_22ago_24sep.columns

Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

### mdt-data abril a junio 2023.csv

In [31]:
doc_mdt_data_abril_junio_2023 = pd.read_csv(tmp_dir+'mdt-data abril a junio 2023.csv',index_col=False)

In [33]:
doc_mdt_data_abril_junio_2023.columns

Index(['list_name', 'itemname', 'material', 'quantity', 'description',
       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
       'log_index'],
      dtype='object')

### mdt-data agosto 2023.csv

In [52]:
doc_mdt_data_agosto_2023 = pd.read_csv(tmp_dir+'mdt-data agosto 2023.csv',index_col=False)

In [54]:
doc_mdt_data_agosto_2023.columns

Index(['list_name', 'itemname', 'material', 'quantity', 'description',
       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
       'log_index'],
      dtype='object')

### mdt-data_27agosto.csv

In [61]:
doc_mdt_data_27agosto = pd.read_csv(tmp_dir+'mdt-data_27agosto.csv',index_col=False)

/tmp/ipykernel_20994/4288113453.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_27agosto = pd.read_csv(tmp_dir+'mdt-data_27agosto.csv',index_col=False)


In [63]:
doc_mdt_data_27agosto.columns

Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

### mdt-data_octubre a marzo 2023.csv

In [64]:
doc_mdt_data_octubre_marzo_2023 = pd.read_csv(tmp_dir+'mdt-data_octubre a marzo 2023.csv',index_col=False)

In [65]:
doc_mdt_data_octubre_marzo_2023.columns

Index(['list_name', 'itemname', 'material', 'quantity', 'description',
       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
       'log_index'],
      dtype='object')

### mdt-data_Todos los tramos_30nov.csv

In [68]:
doc_mdt_data_todos_los_tramos_30nov = pd.read_csv(tmp_dir+'mdt-data_Todos los tramos_30nov.csv',index_col=False)

/tmp/ipykernel_20994/3909461749.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_todos_los_tramos_30nov = pd.read_csv(tmp_dir+'mdt-data_Todos los tramos_30nov.csv',index_col=False)


In [72]:
doc_mdt_data_todos_los_tramos_30nov.columns

Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

## Concatenate

### CSVs type 1 - Containing master_item_id

In [134]:
# csvs type 1 contain 'master_item_id' as a starting column.
# All columns: ['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
#       'latitude', 'longitude', 'altitude', 'description', 'radius',
#       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location']

csvs_type1 = ['29_0ctubre_mdt-data.csv','mdt-data (1).csv','mdt-data 5julio-24sep.csv','mdt-data 22ago-24sep.csv','mdt-data_27agosto.csv','mdt-data_Todos los tramos_30nov.csv']

# Concatenate csvs
type_1 = pd.DataFrame()
for csv in csvs_type1:
    tmp_gdf = pd.read_csv(tmp_dir+csv,
                          index_col=False)
    type_1 = pd.concat([type_1,tmp_gdf])
    
# Set column with date time format
type_1['datetime'] = pd.to_datetime(type_1['timestamp'], format='%Y%m%d%H%M%S')

print(type_1.shape)
type_1.head(1)

/tmp/ipykernel_20994/1124814518.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_20994/1124814518.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_20994/1124814518.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_20994/1124814518.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_20994/1124814518.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/

(15043, 17)


,master_item_id,item_id,itemname,id,log_index,quantity,latitude,longitude,altitude,description,radius,manual_upload,project_id,project_name,timestamp,location,datetime
0,1,3730,Bolsa de plástico para comestibles,2251010,56719,5,25.641495,-100.293776,530.400024,NaN,4.819,0,223.0,Default,20221029140531,Monterrey,2022-10-29 14:05:31


In [209]:
# csvs type 2 contain 'list_name' as a starting column.
# All columns: ['list_name', 'itemname', 'material', 'quantity', 'description',
#       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
#       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
#       'log_index']

csvs_type2 = ['mdt-data abril a junio 2023.csv','mdt-data agosto 2023.csv','mdt-data_octubre a marzo 2023.csv']

# Concatenate csvs
type_2 = pd.DataFrame()
for csv in csvs_type2:
    tmp_gdf = pd.read_csv(tmp_dir+csv,
                          index_col=False)
    type_2 = pd.concat([type_2,tmp_gdf])
    
# Set column with date time format
type_2['datetime'] = pd.to_datetime(type_2['timestamp'], format='%Y%m%d%H%M%S')

print(type_2.shape)
type_2.head(1)

(9016, 19)


,list_name,itemname,material,quantity,description,latitude,longitude,altitude,radius,location,timestamp,dt,project_name,username,manual_upload,event_name,id,log_index,datetime
0,Tecnológico de Monterrey,Bolsa de plástico para comestibles,Productos Plásticos para Comida,10,NaN,25.641958,-100.296648,553.976016,4.743728,"Monterrey, Nuevo León, Mexico",20230415143048,2023-04-15 14:30:48,Default,AlexGuajardo,0,NaN,2398898,61617,2023-04-15 14:30:48


In [221]:
material_itemname_dicc = {'plastico':['Bolsa de plástico para comestibles', 'Bolsa de plástico (otra)','Botella de plástico','Tapa de botella de plástico',
                                      'Vasos o tapas de unicel o plástico','Envoltorio de plástico para alimentos','Fragmentos de plástico duro', 'Fragmentos de película de plástico',
                                      'Utensilios de plástico','Otro plástico relacionado con los alimentos','Otro plástico','Otros fragmentos',
                                      'Cuerda de plástico/ cinta o correas de embalaje','Bolsas a granel','Popotes','Chancletas/sandalias','Sedal','Cuerda de plástico', 
                                      'Red de plástico o piezas de red','Tazón de comida para llevar','Plastico para cerveza'
                                     ],
                          'tabaco':['Cajetilla de cigarrillos', 'Otro producto de tabaco','Cigarrillos','Sobres de tabaco'],
                          'metal':['Latas de aluminio o estaño','Tapas o lengüetas de metal para botellas','Fragmentos de metal','Papel de aluminio','Otro metal'],
                          'vidrio':['Botella de vidrio','Otro vidrio'],
                          'llantas':['Llantas'],
                          'tela':['Ropa','Piezas de tela','Otra tela','Toallas o trapos','Ropa o zapatos'],
                          'escombros':['Tablas de madera', 'Materiales de construcción','Agregado y ladrillo','Otros artículos de construcción','Pernos/ clavos y tornillos'],
                          'papel':['Envoltura de alimentos de papel sin recubrimiento','Caja de papel multimaterial','Papel','Recibos','Otro papel','Bolsas de papel'],
                          'carton':['Cartón acanalado','Cartón recubierto'],
                          'otros':['Otro','Boyas y Flotadores'],
                          'salud e higiene':['Otro producto de cuidado personal','Redes para el cabello','Tubo de pasta de dientes u otro producto','Champú u otro recipiente de HDPE',
                                             'Cotonetes de algodón','Cepillos de dientes','Embalaje de mascarilla facial','Sobre de productos de cuidado personal'],
                          'unicel':['Contenedor de espuma de poliestireno (unicel)','Fragmentos de espuma (unicel)'],
                          'electronica':['Otros desechos electrónicos','Fragmentos de electrónicos'],
                          'organico':['Otros desechos orgánicos','Desechos de comida'],
                          'ceramica':['Fragmentos de vidrio o cerámica'],
                          'comercio':['Tienda de comestibles','Tienda al por menor','Otra tienda','Tienda de conveniencia','Puesto de comida para llevar','Tienda de café / té',
                                      'Otro Restaurante'],
                          'EPI':['Máscaras faciales', 'Careta','Elemento de prueba','Cubrezapatos de hospital','Otros EPI','Embalaje de EPI asociado','Guantes desechables',
                                'Paquete de medicamentos','Toallitas desinfectantes'],
                          'infraestructura':['Bote de la basura', 'Bote de reciclaje','Canal', 'Reciclaje informal', 'Colector de aguas pluviales','Basurero'],
                          'baterias':['Baterías',],
                          'goma':['Ligas de goma'],
                          'madera':['Palito de paleta'],
                          'pesca':['Otro equipo de pesca'],
                          'cenizas':['Pila de cenizas']}

In [222]:
# Revisar que todos los itemnames estén asignados a un material en el diccionario:

analized_items = []
for k in material_itemname_dicc.keys():
    for item in material_itemname_dicc[k]:
        analized_items.append(item)
        
for item in list(type_1.itemname.unique()):
    if item not in analized_items:
        print(item)       
        
for item in list(type_2.itemname.unique()):
    if item not in analized_items:
        print(item)
        
for item in list(residuos_point.itemname.unique()):
    if item not in analized_items:
        print(item)    

# Old code

In [3]:
columns_tokeep = ['itemname','tipo','material','quantity','latitude','longitude','altitude','data_origin']

### Batch 1

In [4]:
doc_9_de_enero = pd.read_csv(tmp_dir+'9 de enero.csv')
doc_9_de_enero['data_origin'] = '9 de enero.csv'

doc_27_05 = pd.read_csv(tmp_dir+'27_05.csv')
doc_27_05['data_origin'] = '27_05.csv'

doc_30_05 = pd.read_csv(tmp_dir+'30_05.csv')
doc_30_05['data_origin'] = '30_05.csv'

doc_todo_hasta_septiembre = pd.read_csv(tmp_dir+'todo hasta septiembre 24.csv')
doc_todo_hasta_septiembre['data_origin'] = 'todo hasta septiembre 24.csv'

#### Data treatment - 9 de enero.csv

In [5]:
doc_9_de_enero.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,Unnamed: 6,data_origin
0,Latas de aluminio o estaño,Aluminio,4,0.084465,-79.050839,627.300049,NaN,9 de enero.csv


In [6]:
doc_9_de_enero_f = doc_9_de_enero.copy()
doc_9_de_enero_f['tipo'] = np.nan
doc_9_de_enero_f.rename(columns={'Category':'material'},inplace=True)
doc_9_de_enero_f = doc_9_de_enero_f[columns_tokeep]

# Show
print(doc_9_de_enero_f.shape)
doc_9_de_enero_f.head(2)

(10237, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,4,0.084465,-79.050839,627.300049,9 de enero.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,-33.629462,-71.549508,104.600000,9 de enero.csv


#### Data treatment - 27_05.csv

In [7]:
doc_27_05.head(1)

,itemname,material,id,quantity,description,latitude,longitude,altitude,data_origin
0,Tablas de madera,Materiales de Construcción,Construccion,2,NaN,25.639195,-100.303704,544.800049,27_05.csv


In [8]:
doc_27_05_f = doc_27_05.copy()
doc_27_05_f.rename(columns={'material':'tipo',
                            'id':'material'},inplace=True)
doc_27_05_f = doc_27_05_f[columns_tokeep]

# Show
print(doc_27_05_f.shape)
doc_27_05_f.head(2)

(1484, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Tablas de madera,Materiales de Construcción,Construccion,2,25.639195,-100.303704,544.800049,27_05.csv
1,Tablas de madera,Materiales de Construcción,Construccion,5,25.639991,-100.302126,540.700012,27_05.csv


#### Data treatment - 30_05.csv

In [9]:
doc_30_05.head(1)

,itemname,material,id,quantity,description,latitude,longitude,altitude,data_origin
0,Other Metal,METAL,Metal,1,NaN,25.641804,-100.314824,562.835693,30_05.csv


In [10]:
doc_30_05_f = doc_30_05.copy()
doc_30_05_f.rename(columns={'material':'tipo',
                            'id':'material'},inplace=True)
doc_30_05_f = doc_30_05_f[columns_tokeep]

# Show
print(doc_30_05_f.shape)
doc_30_05_f.head(2)

(315, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Other Metal,METAL,Metal,1,25.641804,-100.314824,562.835693,30_05.csv
1,Other Metal,METAL,Metal,1,25.641804,-100.314824,562.835693,30_05.csv


#### Data treatment - todo_hasta_septiembre_24.csv

In [11]:
doc_todo_hasta_septiembre.head(1)

,quantity,category,T,latitude,longitude,altitude,data_origin
0,1,plastico,Bolsa de plastico para comestibles,25.643146,-100.312009,606.27422,todo hasta septiembre 24.csv


In [12]:
doc_todo_hasta_septiembre_f = doc_todo_hasta_septiembre.copy()
doc_todo_hasta_septiembre_f['tipo'] = np.nan
doc_todo_hasta_septiembre_f.rename(columns={'T':'itemname',
                                            'category':'material'},inplace=True)
doc_todo_hasta_septiembre_f = doc_todo_hasta_septiembre_f[columns_tokeep]

# Show
print(doc_todo_hasta_septiembre_f.shape)
doc_todo_hasta_septiembre_f.head(2)

(5166, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plastico para comestibles,NaN,plastico,1,25.643146,-100.312009,606.274220,todo hasta septiembre 24.csv
1,Bolsa de plastico para comestibles,NaN,plastico,1,25.643812,-100.310577,626.850122,todo hasta septiembre 24.csv


### Batch 2

In [13]:
doc_2022_2023 = pd.read_csv(tmp_dir+'2022_2023_fixcoord.csv')
doc_2022_2023['data_origin'] = '2022_2023.csv'

doc_punto1 = pd.read_csv(tmp_dir+'punto1.csv')
doc_punto1['data_origin'] = 'punto1.csv'

doc_punto_2 = pd.read_csv(tmp_dir+'punto 2.csv')
doc_punto_2['data_origin'] = 'punto 2.csv'

doc_punto_3 = pd.read_csv(tmp_dir+'punto 3.csv')
doc_punto_3['data_origin'] = 'punto 3.csv'

doc_punto_4 = pd.read_csv(tmp_dir+'punto 4.csv')
doc_punto_4['data_origin'] = 'punto 4.csv'

doc_punto_5 = pd.read_csv(tmp_dir+'punto 5.csv')
doc_punto_5['data_origin'] = 'punto 5.csv'

doc_punto_6 = pd.read_csv(tmp_dir+'punto 6.csv')
doc_punto_6['data_origin'] = 'punto 6.csv'

doc_punto_7 = pd.read_csv(tmp_dir+'punto 7.csv')
doc_punto_7['data_origin'] = 'punto 7.csv'

doc_punto_8 = pd.read_csv(tmp_dir+'punto 8.csv')
doc_punto_8['data_origin'] = 'punto 8.csv'

#### Data treatment - punto1.csv

In [15]:
doc_punto1.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Latas de aluminio o estaño,Aluminio,11,25.644154,-100.310618,627.104902,NaN,punto1.csv


In [16]:
doc_punto1_f = doc_punto1.copy()
doc_punto1_f['tipo'] = np.nan
doc_punto1_f.rename(columns={'Category':'material'},inplace=True)
doc_punto1_f = doc_punto1_f[columns_tokeep]

# Show
print(doc_punto1_f.shape)
doc_punto1_f.head(2)

(339, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,11,25.644154,-100.310618,627.104902,punto1.csv
1,Papel de aluminio,NaN,Aluminio,6,25.644065,-100.310389,634.796484,punto1.csv


#### Data treatment - punto 2.csv

In [17]:
doc_punto_2.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Latas de aluminio o estaño,Aluminio,1,25.641301,-100.312949,558.200012,NaN,punto 2.csv


In [18]:
doc_punto_2_f = doc_punto_2.copy()
doc_punto_2_f['tipo'] = np.nan
doc_punto_2_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_2_f = doc_punto_2_f[columns_tokeep]

# Show
print(doc_punto_2_f.shape)
doc_punto_2_f.head(2)

(374, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,1,25.641301,-100.312949,558.200012,punto 2.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,25.641301,-100.312769,573.357257,punto 2.csv


#### Data treatment - punto 3.csv

In [19]:
doc_punto_3.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Cartón recubierto,Carton,1,25.641006,-100.310754,573.351175,NaN,punto 3.csv


In [20]:
doc_punto_3_f = doc_punto_3.copy()
doc_punto_3_f['tipo'] = np.nan
doc_punto_3_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_3_f = doc_punto_3_f[columns_tokeep]

# Show
print(doc_punto_3_f.shape)
doc_punto_3_f.head(2)

(180, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Cartón recubierto,NaN,Carton,1,25.641006,-100.310754,573.351175,punto 3.csv
1,Cartón recubierto,NaN,Carton,1,25.640935,-100.310646,573.351175,punto 3.csv


#### Data treatment - punto 4.csv

In [21]:
doc_punto_4.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Latas de aluminio o estaño,Aluminio,1,25.639487,-100.309424,572.535219,NaN,punto 4.csv


In [22]:
doc_punto_4_f = doc_punto_4.copy()
doc_punto_4_f['tipo'] = np.nan
doc_punto_4_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_4_f = doc_punto_4_f[columns_tokeep]

# Show
print(doc_punto_4_f.shape)
doc_punto_4_f.head(2)

(61, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,1,25.639487,-100.309424,572.535219,punto 4.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,25.639592,-100.309716,573.351175,punto 4.csv


#### Data treatment - punto 5.csv

In [23]:
doc_punto_5.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Latas de aluminio o estaño,Aluminio,1,25.640303,-100.30339,562.19939,NaN,punto 5.csv


In [24]:
doc_punto_5_f = doc_punto_5.copy()
doc_punto_5_f['tipo'] = np.nan
doc_punto_5_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_5_f = doc_punto_5_f[columns_tokeep]

# Show
print(doc_punto_5_f.shape)
doc_punto_5_f.head(2)

(416, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,1,25.640303,-100.303390,562.199390,punto 5.csv
1,Papel de aluminio,NaN,Aluminio,1,25.640279,-100.303387,562.986032,punto 5.csv


#### Data treatment - punto 6.csv

In [25]:
doc_punto_6.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Latas de aluminio o estaño,Aluminio,1,25.641152,-100.293677,530.300049,NaN,punto 6.csv


In [26]:
doc_punto_6_f = doc_punto_6.copy()
doc_punto_6_f['tipo'] = np.nan
doc_punto_6_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_6_f = doc_punto_6_f[columns_tokeep]

# Show
print(doc_punto_6_f.shape)
doc_punto_6_f.head(2)

(450, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,1,25.641152,-100.293677,530.300049,punto 6.csv
1,Papel de aluminio,NaN,Aluminio,1,25.640885,-100.293805,534.300049,punto 6.csv


#### Data treatment - punto 7.csv

In [27]:
doc_punto_7.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Latas de aluminio o estaño,Aluminio,1,25.656683,-100.283225,526.301173,NaN,punto 7.csv


In [28]:
doc_punto_7_f = doc_punto_7.copy()
doc_punto_7_f['tipo'] = np.nan
doc_punto_7_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_7_f = doc_punto_7_f[columns_tokeep]

# Show
print(doc_punto_7_f.shape)
doc_punto_7_f.head(2)

(148, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,1,25.656683,-100.283225,526.301173,punto 7.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,25.656664,-100.283432,526.532977,punto 7.csv


#### Data treatment - punto 8.csv

In [29]:
doc_punto_8.head(1)

,itemname,Category,quantity,latitude,longitude,altitude,field_7,data_origin
0,Papel de aluminio,Aluminio,1,25.637287,-100.305609,571.355888,NaN,punto 8.csv


In [30]:
doc_punto_8_f = doc_punto_8.copy()
doc_punto_8_f['tipo'] = np.nan
doc_punto_8_f.rename(columns={'Category':'material'},inplace=True)
doc_punto_8_f = doc_punto_8_f[columns_tokeep]

# Show
print(doc_punto_8_f.shape)
doc_punto_8_f.head(2)

(499, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Papel de aluminio,NaN,Aluminio,1,25.637287,-100.305609,571.355888,punto 8.csv
1,Papel de aluminio,NaN,Aluminio,1,25.637121,-100.305543,571.415097,punto 8.csv


#### Data treatment - 2022_2023.csv

In [31]:
doc_2022_2023.head(1)

,itemname,material,Column1,quantity,latitude,longitude,altitude,radius,Column2,Column3,Column4,Column5,lat_fix,lon_fix,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,2564314581,-1003120089,6.062742e+09,4.710354e+09,25.643146,-100.312009,2564314581,-1003120089,25.643146,-100.312009,2022_2023.csv


In [32]:
# Additional internal cleaning. This csv is different.
internal_cleaning = ['itemname','material','Column1','quantity','lat_fix','lon_fix','altitude','data_origin']
doc_2022_2023_f = doc_2022_2023[internal_cleaning]

doc_2022_2023_f.rename(columns={'lat_fix':'latitude',
                              'lon_fix':'longitude'},inplace=True)


doc_2022_2023_f2 = doc_2022_2023_f.loc[((doc_2022_2023_f.latitude < 26) & (doc_2022_2023_f.latitude > 25)) &
                                       ((doc_2022_2023_f.longitude < -100) & (doc_2022_2023_f.latitude > -101))]

# Regular formatting
doc_2022_2023_f3 = doc_2022_2023_f2.copy()
doc_2022_2023_f3.rename(columns={'material':'tipo',
                                'Column1':'material'},inplace=True)
doc_2022_2023_f3 = doc_2022_2023_f3[columns_tokeep]

# Show
print(doc_2022_2023_f3.shape)
doc_2022_2023_f3.head(2)

(12360, 8)


/tmp/ipykernel_1256/3375308736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_2022_2023_f.rename(columns={'lat_fix':'latitude',


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.643146,-100.312009,6.062742e+09,2022_2023.csv
1,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.643812,-100.310577,6.268501e+09,2022_2023.csv


### Batch 3

In [33]:
doc_01_01 = pd.read_csv(tmp_dir+'01_01.csv')
doc_01_01['data_origin'] = '01_01.csv'

doc_01_02 = pd.read_csv(tmp_dir+'01_02.csv')
doc_01_02['data_origin'] = '01_02.csv'

doc_02_01 = pd.read_csv(tmp_dir+'02_01.csv')
doc_02_01['data_origin'] = '02_01.csv'

doc_02_02 = pd.read_csv(tmp_dir+'02_02.csv')
doc_02_02['data_origin'] = '02_02.csv'

doc_03_01 = pd.read_csv(tmp_dir+'03_01.csv')
doc_03_01['data_origin'] = '03_01.csv'

doc_03_02 = pd.read_csv(tmp_dir+'03_02.csv')
doc_03_02['data_origin'] = '03_02.csv'

#### Data treatment - 01_01.csv

In [35]:
doc_01_01.head(1)

,itemname,material,tipo,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,1,25.63975,-100.302499,570.187852,01_01.csv


In [36]:
doc_01_01_f = doc_01_01.copy()
doc_01_01_f.rename(columns={'material':'tipo',
                            'tipo':'material'},inplace=True)
doc_01_01_f = doc_01_01_f[columns_tokeep]

# Show
print(doc_01_01_f.shape)
doc_01_01_f.head(2)

(886, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,1,25.639750,-100.302499,570.187852,01_01.csv
1,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,1,25.639572,-100.303028,566.861935,01_01.csv


#### Data treatment - 01_02.csv

In [37]:
doc_01_02.head(1)

,itemname,material,tipo,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.638845,-100.304293,546.100037,01_02.csv


In [38]:
doc_01_02_f = doc_01_02.copy()
doc_01_02_f.rename(columns={'material':'tipo',
                            'tipo':'material'},inplace=True)
doc_01_02_f = doc_01_02_f[columns_tokeep]

# Show
print(doc_01_02_f.shape)
doc_01_02_f.head(2)

(2093, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.638845,-100.304293,546.100037,01_02.csv
1,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.638845,-100.304293,546.100037,01_02.csv


#### Data treatment - 02_01.csv

In [39]:
doc_02_01.head(1)

,itemname,material,tipo,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.641024,-100.310774,574.64481,02_01.csv


In [40]:
doc_02_01_f = doc_02_01.copy()
doc_02_01_f.rename(columns={'material':'tipo',
                            'tipo':'material'},inplace=True)
doc_02_01_f = doc_02_01_f[columns_tokeep]

# Show
print(doc_02_01_f.shape)
doc_02_01_f.head(2)

(991, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.641024,-100.310774,574.644810,02_01.csv
1,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,1,25.640954,-100.310654,573.157998,02_01.csv


#### Data treatment - 02_02.csv

In [41]:
doc_02_02.head(1)

,itemname,material,tipo,quantity,description,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,8,NaN,25.639855,-100.309363,551.900024,02_02.csv


In [42]:
doc_02_02_f = doc_02_02.copy()
doc_02_02_f.rename(columns={'material':'tipo',
                            'tipo':'material'},inplace=True)
doc_02_02_f = doc_02_02_f[columns_tokeep]

# Show
print(doc_02_02_f.shape)
doc_02_02_f.head(2)

(57, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,8,25.639855,-100.309363,551.900024,02_02.csv
1,Bolsa de Plastico (otra),Productos Plasticos para Comida,Plastico,35,25.640555,-100.310213,553.100037,02_02.csv


#### Data treatment - 03_01.csv

In [43]:
doc_03_01.head(1)

,itemname,material,tipo,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,5,25.641495,-100.293776,530.400024,03_01.csv


In [44]:
doc_03_01_f = doc_03_01.copy()
doc_03_01_f.rename(columns={'material':'tipo',
                            'tipo':'material'},inplace=True)
doc_03_01_f = doc_03_01_f[columns_tokeep]

# Show
print(doc_03_01_f.shape)
doc_03_01_f.head(2)

(342, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,5,25.641495,-100.293776,530.400024,03_01.csv
1,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,17,25.641586,-100.293740,530.700012,03_01.csv


#### Data treatment - 03_02.csv

In [45]:
doc_03_02.head(1)

,itemname,material,tipo,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,10,25.641958,-100.296649,553.976017,03_02.csv


In [46]:
doc_03_02_f = doc_03_02.copy()
doc_03_02_f.rename(columns={'material':'tipo',
                            'tipo':'material'},inplace=True)
doc_03_02_f = doc_03_02_f[columns_tokeep]

# Show
print(doc_03_02_f.shape)
doc_03_02_f.head(2)

(632, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,10,25.641958,-100.296649,553.976017,03_02.csv
1,Bolsa de Plastico para comestibles,Productos Plasticos para Comida,Plastico,4,25.642008,-100.296322,550.018779,03_02.csv


### Batch 4

In [47]:
doc_mdt_abril_junio_2023 = pd.read_csv(tmp_dir+'mdt-data abril a junio 2023.csv')
doc_mdt_abril_junio_2023['data_origin'] = 'mdt-abril_junio_2023.csv'

doc_mdt_agosto_2023 = pd.read_csv(tmp_dir+'mdt-data agosto 2023.csv')
doc_mdt_agosto_2023['data_origin'] = 'mdt-agosto_2023.csv'

#### Data treatment - mdt-data abril a junio 2023.csv

In [48]:
doc_mdt_abril_junio_2023.head(1)

,list_name,itemname,material,quantity,description,latitude,longitude,altitude,radius,location,timestamp,dt,project_name,username,manual_upload,event_name,id,log_index,data_origin
0,Tecnológico de Monterrey,Bolsa de plástico para comestibles,Productos Plásticos para Comida,10,NaN,25.641958,-100.296648,553.976016,4.743728,"Monterrey, Nuevo León, Mexico",20230415143048,2023-04-15 14:30:48,Default,AlexGuajardo,0,NaN,2398898,61617,mdt-abril_junio_2023.csv


In [49]:
# Regular formatting
doc_mdt_abril_junio_2023_f = doc_mdt_abril_junio_2023.copy()
doc_mdt_abril_junio_2023_f.rename(columns={'material':'tipo'},inplace=True)
doc_mdt_abril_junio_2023_f['material'] = np.nan
doc_mdt_abril_junio_2023_f = doc_mdt_abril_junio_2023_f[columns_tokeep]

# Show
print(doc_mdt_abril_junio_2023_f.shape)
doc_mdt_abril_junio_2023_f.head(2)

(2392, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plástico para comestibles,Productos Plásticos para Comida,NaN,10,25.641958,-100.296648,553.976016,mdt-abril_junio_2023.csv
1,Bolsa de plástico para comestibles,Productos Plásticos para Comida,NaN,4,25.642008,-100.296322,550.018779,mdt-abril_junio_2023.csv


In [50]:
# Additional cleaning. This csv does not have a "material" column, it must be created from "tipo"
doc_mdt_abril_junio_2023_f.tipo.unique()

array(['Productos Plásticos para Comida', 'Otros Productos Plásticos',
       'Fragmentos de Plástico  ', 'Productos de Cuidado Personal',
       'Productos de Tabaco', 'Materiales de Construcción',
       'Desechos Electrónicos', 'Desechos Orgánicos', 'Metales', 'Vidrio',
       'Tela', 'Papel', 'Herramientas de Pesca', 'Otro', 'Restaurantes',
       'Tienda', 'Infraestructura para Desechos '], dtype=object)

In [51]:
# Additional cleaning. This csv does not have a "material" column, it must be created from "tipo"
doc_mdt_abril_junio_2023_f2 = doc_mdt_abril_junio_2023_f.copy()

fix_names = {'Plastico':['Productos Plásticos para Comida','Otros Productos Plásticos','Fragmentos de Plástico  '],
             'Tabaco':['Productos de Tabaco'],
             'Pesca':['Herramientas de Pesca'],
             'Salud e higiene':['Productos de Cuidado Personal'],
             'Tela':['Tela'],
             'Escombros':['Materiales de Construcción'],
             'Papel':['Papel'],
             'Comercio':['Restaurantes','Tienda'],
             'Electronica':['Desechos Electrónicos'],
             'Organico':['Desechos Orgánicos'],
             'Otros':['Otro'],
             'Vidrio':['Vidrio'],
             'Metal':['Metales'],
             'Recoleccion':['Infraestructura para Desechos ']}

for k in fix_names.keys():
    for item in fix_names[k]:
        idx = doc_mdt_abril_junio_2023_f2.tipo == item
        doc_mdt_abril_junio_2023_f2.loc[idx,'material'] = k
        
# Show
print(doc_mdt_abril_junio_2023_f2.shape)
doc_mdt_abril_junio_2023_f2.head(2)

(2392, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,10,25.641958,-100.296648,553.976016,mdt-abril_junio_2023.csv
1,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,4,25.642008,-100.296322,550.018779,mdt-abril_junio_2023.csv


#### Data treatment - mdt-data agosto 2023.csv

In [53]:
doc_mdt_agosto_2023.head(1)

,list_name,itemname,material,quantity,description,latitude,longitude,altitude,radius,location,timestamp,dt,project_name,username,manual_upload,event_name,id,log_index,data_origin
0,Tecnológico de Monterrey,Bolsa de plástico para comestibles,Productos Plásticos para Comida,2,NaN,25.649795,-100.284015,534.115751,4.730886,"Monterrey, Nuevo León, Mexico",20230819162919,2023-08-19 16:29:19,Default,raulherrera,0,NaN,2589766,68725,mdt-agosto_2023.csv


In [54]:
# Regular formatting
doc_mdt_agosto_2023_f = doc_mdt_agosto_2023.copy()
doc_mdt_agosto_2023_f.rename(columns={'material':'tipo'},inplace=True)
doc_mdt_agosto_2023_f['material'] = np.nan
doc_mdt_agosto_2023_f = doc_mdt_agosto_2023_f[columns_tokeep]

# Show
print(doc_mdt_agosto_2023_f.shape)
doc_mdt_agosto_2023_f.head(2)

(268, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plástico para comestibles,Productos Plásticos para Comida,NaN,2,25.649795,-100.284015,534.115751,mdt-agosto_2023.csv
1,Bolsa de plástico para comestibles,Productos Plásticos para Comida,NaN,2,25.649796,-100.282770,532.783177,mdt-agosto_2023.csv


In [55]:
# Additional cleaning. This csv does not have a "material" column, it must be created from "tipo"
doc_mdt_agosto_2023_f.tipo.unique()

array(['Productos Plásticos para Comida', 'Otros Productos Plásticos',
       'Fragmentos de Plástico  ', 'Productos de Cuidado Personal',
       'Productos de Tabaco', 'Materiales de Construcción',
       'Desechos Orgánicos', 'Metales', 'Vidrio', 'Tela', 'Papel', 'EPI',
       'Restaurantes', 'Tienda', 'Hidrología',
       'Infraestructura para Desechos '], dtype=object)

In [56]:
# Additional cleaning. This csv does not have a "material" column, it must be created from "tipo"
doc_mdt_agosto_2023_f2 = doc_mdt_agosto_2023_f.copy()

fix_names = {'Plastico':['Productos Plásticos para Comida','Otros Productos Plásticos','Fragmentos de Plástico  '],
             'Tabaco':['Productos de Tabaco'],
             'Salud e higiene':['Productos de Cuidado Personal'],
             'Tela':['Tela'],
             'Escombros':['Materiales de Construcción'],
             'Papel':['Papel'],
             'Comercio':['Restaurantes','Tienda'],
             'Organico':['Desechos Orgánicos'],
             'Hidrología':['Hidrología'],
             'Vidrio':['Vidrio'],
             'Metal':['Metales'],
             'Recoleccion':['Infraestructura para Desechos '],
             'EPI':['EPI']}

for k in fix_names.keys():
    for item in fix_names[k]:
        idx = doc_mdt_agosto_2023_f2.tipo == item
        doc_mdt_agosto_2023_f2.loc[idx,'material'] = k
        
# Show
print(doc_mdt_agosto_2023_f2.shape)
doc_mdt_agosto_2023_f2.head(2)

(268, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,2,25.649795,-100.284015,534.115751,mdt-agosto_2023.csv
1,Bolsa de plástico para comestibles,Productos Plásticos para Comida,Plastico,2,25.649796,-100.282770,532.783177,mdt-agosto_2023.csv


## Concatenate

In [58]:
all_together = pd.concat([doc_9_de_enero_f,
                          doc_27_05_f,
                          doc_30_05_f,
                          doc_todo_hasta_septiembre_f,
                          doc_2022_2023_f3,
                          doc_punto1_f,
                          doc_punto_2_f,
                          doc_punto_3_f,
                          doc_punto_4_f,
                          doc_punto_5_f,
                          doc_punto_6_f,
                          doc_punto_7_f,
                          doc_punto_8_f,
                          doc_01_01_f,
                          doc_01_02_f,
                          doc_02_01_f,
                          doc_02_02_f,
                          doc_03_01_f,
                          doc_03_02_f,
                          doc_mdt_abril_junio_2023_f2,
                          doc_mdt_agosto_2023_f2], ignore_index = True)
                         
# Show
print(all_together.shape)
all_together.head(2)

(39690, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,4,0.084465,-79.050839,627.300049,9 de enero.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,-33.629462,-71.549508,104.600000,9 de enero.csv


## Fix badly named columns

### Fix column material

In [60]:
all_together.material.unique()

array(['Aluminio', 'Baterias', 'Carton', 'Cenizas', 'Ceramica',
       'Colector de aguas pluviales', 'Comercio', 'Electronica',
       'Escombros', 'Goma', 'Llantas', 'Madera', 'Metal', 'Organico',
       'Otros', 'Papel', 'Pesca', 'Plastico', 'Recoleccion',
       'Salud e higiene', 'Tabaco', 'Tela', 'Unicel', 'Vidrio',
       'Construccion', 'plastico', 'tabaco', 'unicel', 'EPI',
       'salud e higiene', 'tela', 'escombros', 'papel', 'carton',
       'Recipiente de basura', 'Reciclaje', 'electronicos', 'organico',
       'ceramica', 'otros', 'Canal', 'Colector', 'Basurero', 'Metales',
       'Hidrología'], dtype=object)

In [61]:
all_together_fix = all_together.copy()

fix_names = {'Plastico':['plastico'],
             'Tabaco':['tabaco'],
             'Unicel':['unicel'],
             'Salud e higiene':['salud e higiene'],
             'Tela':['tela'],
             'Escombros':['escombros','Construccion'],
             'Papel':['papel'],
             'Carton':['carton'],
             'Electronica':['electronicos'],
             'Organico':['organico'],
             'Ceramica':['ceramica'],
             'Otros':['otros'],
             'Metal':['Metales']}

for k in fix_names.keys():
    for item in fix_names[k]:
        idx = all_together_fix.material == item
        all_together_fix.loc[idx,'material'] = k

In [62]:
all_together_fix.material.unique()

array(['Aluminio', 'Baterias', 'Carton', 'Cenizas', 'Ceramica',
       'Colector de aguas pluviales', 'Comercio', 'Electronica',
       'Escombros', 'Goma', 'Llantas', 'Madera', 'Metal', 'Organico',
       'Otros', 'Papel', 'Pesca', 'Plastico', 'Recoleccion',
       'Salud e higiene', 'Tabaco', 'Tela', 'Unicel', 'Vidrio', 'EPI',
       'Recipiente de basura', 'Reciclaje', 'Canal', 'Colector',
       'Basurero', 'Hidrología'], dtype=object)

### Fix column itemname

In [63]:
all_together_fix.itemname.unique()

array(['Latas de aluminio o estaño', 'Papel de aluminio', 'Baterías',
       'Cartón acanalado', 'Cartón recubierto',
       'Envoltura de alimentos de papel sin recubrimiento',
       'Pila de cenizas', 'Fragmentos de vidrio o cerámica',
       'Colector de aguas pluviales', 'Puesto de comida para llevar',
       'Tienda al por menor', 'Tienda de comestibles',
       'Tienda de conveniencia', 'Fragmentos de electrónicos',
       'Otros desechos electrónicos', 'Agregado y ladrillo',
       'Materiales de construcción', 'Otros artículos de construcción',
       'Otros fragmentos', 'Ligas de goma', 'Llantas', 'Palito de paleta',
       'Tablas de madera', 'Fragmentos de metal', 'Otro metal',
       'Pernos/ clavos y tornillos',
       'Tapas o lengüetas de metal para botellas',
       'Otros desechos orgánicos', 'Desechos de comida',
       'Boyas y Flotadores', 'Otro', 'Sedal',
       'Caja de papel multimaterial', 'Otro papel', 'Papel', 'Recibos',
       'Otro equipo de pesca', 'Bolsa 

In [64]:
all_together_fix_2 = all_together_fix.copy()

fix_names = {'Latas de aluminio o estaño':['Aluminum or tin cans'],
             'Papel de aluminio':['papel de aluminio'],
             'Baterías':['baterias'],
             'Cartón acanalado':['carton acanalado'],
             'Cartón recubierto':['carton recubierto'],
             'Envoltura de alimentos de papel sin recubrimiento':['Food wrappers (paper)'],
             'Fragmentos de vidrio o cerámica':['Fragmentos de vidrio o cerÃ¡mica','Glass fragments'],
             'Fragmentos de electrónicos':['Fragmentos de electronicos'],
             'Otros desechos electrónicos':['Otros desechos electronicos'],
             'Materiales de construcción':['Materiales de cconstruccion','Materiales de Construccion','Lumber/Building Materials'],
             'Otros artículos de construcción':['Otros artÃ­culos de cconstruccion','Otros artículos de Construccion','Otros articulos de Construccion'],
             'Palito de paleta':['Pallets'],
             'Fragmentos de metal':['Metal fragments'],
             'Otro metal':['Other Metal'],
             'Tapas o lengüetas de metal para botellas':['Tapas o lenguetas de metal para botellas'],
             'Otros desechos orgánicos':['Otros desechos organico'],
             'Bolsas de papel':['Paper Bags'],
             'Papel':['papel','Paper and Cardboard'],
             'Bolsa de plástico (otra)':['Bolsa de plastico (otra)','Bolsa de Plastico (otra)'],
             'Bolsa de plástico para comestibles':['Bolsa de plastico para comestibles','Bolsa de Plastico para comestibles'],
             'Botella de plástico':['Botella de plastico','Botella de Plastico'],
             'Cuerda de plástico':['Cuerda de plastico'],
             'Cuerda de plástico/ cinta o correas de embalaje':['Cuerda de plastico/ cinta o correas de embalaje','Cuerda de Plastico/ cinta o correas de embalaje'],
             'Envoltorio de plástico para alimentos':['Envoltorio de plastico para alimentos','Envoltorio de Plastico para alimentos'],
             'Fragmentos de película de plástico':['Fragmentos de peliculade plastico','Fragmentos de película de Plastico'],
             'Fragmentos de plástico duro':['Fragmentos de plastico duro','Fragmentos de Plastico duro'],
             'Otro plástico':['Otro plastico','Otro Plastico'],
             'Otro plástico relacionado con los alimentos':['Otro plastico relacionado con los alimentos','Otro Plastico relacionado con los alimentos'],
             'Red de plástico o piezas de red':['Red de plastico o piezas de red','Tapa de botella de Plastico'],
             'Tapa de botella de plástico':['Tapa de botella de plastico'],
             'Utensilios de plástico':['Utensilios de plastico','Utensilios de Plastico'],
             'Champú u otro recipiente de HDPE':['ChampÃº u otro recipiente de HDPE'],
             'Cotonetes de algodón':['Cotonetes de algodÃ³n'],
             'Máscaras faciales':['MÃ¡scaras faciales','Fabric Face Masks'],
             'Otra tela':['Other Cloth'],
             'Piezas de tela':['Fabric pieces'],
             'Tazón de comida para llevar':['TazÃ³n de comida para llevar'],
             'Vasos o tapas de unicel o plástico':['Vasos o tapas de unicel o plastico','Vasos o tapas de unicel o Plastico'],
             'Botella de vidrio':['Glass Jars'],
             'Otro papel':['Other Paper & Lumber'],
             'Bolsas de papel':['Paper Bags'],
             'Toallas o trapos':['Towels or rags'],
             'Ropa o zapatos':['Clothing and Shoes'],
             'Tienda de café / té':['Tienda de cafÃ© / tÃ©']
            }

for k in fix_names.keys():
    for item in fix_names[k]:
        idx = all_together_fix_2.itemname == item
        all_together_fix_2.loc[idx,'itemname'] = k

In [65]:
all_together_fix_2.itemname.unique()

array(['Latas de aluminio o estaño', 'Papel de aluminio', 'Baterías',
       'Cartón acanalado', 'Cartón recubierto',
       'Envoltura de alimentos de papel sin recubrimiento',
       'Pila de cenizas', 'Fragmentos de vidrio o cerámica',
       'Colector de aguas pluviales', 'Puesto de comida para llevar',
       'Tienda al por menor', 'Tienda de comestibles',
       'Tienda de conveniencia', 'Fragmentos de electrónicos',
       'Otros desechos electrónicos', 'Agregado y ladrillo',
       'Materiales de construcción', 'Otros artículos de construcción',
       'Otros fragmentos', 'Ligas de goma', 'Llantas', 'Palito de paleta',
       'Tablas de madera', 'Fragmentos de metal', 'Otro metal',
       'Pernos/ clavos y tornillos',
       'Tapas o lengüetas de metal para botellas',
       'Otros desechos orgánicos', 'Desechos de comida',
       'Boyas y Flotadores', 'Otro', 'Sedal',
       'Caja de papel multimaterial', 'Otro papel', 'Papel', 'Recibos',
       'Otro equipo de pesca', 'Bolsa 

## Delete categories that will not be used in analysis

In [66]:
all_together_fix_2.material.unique()

array(['Aluminio', 'Baterias', 'Carton', 'Cenizas', 'Ceramica',
       'Colector de aguas pluviales', 'Comercio', 'Electronica',
       'Escombros', 'Goma', 'Llantas', 'Madera', 'Metal', 'Organico',
       'Otros', 'Papel', 'Pesca', 'Plastico', 'Recoleccion',
       'Salud e higiene', 'Tabaco', 'Tela', 'Unicel', 'Vidrio', 'EPI',
       'Recipiente de basura', 'Reciclaje', 'Canal', 'Colector',
       'Basurero', 'Hidrología'], dtype=object)

In [82]:
# Create a "checker" column. All rows will be kept if checker=1. Categories to be deleted are assigned checker=0.
all_together_fix_2['checker'] = 1

# Assign checker=0
del_categories = ['Colector de aguas pluviales','Comercio','Recoleccion','Recipiente de basura', 'Reciclaje', 'Canal', 'Colector','Basurero', 'Hidrología']
for mat in del_categories:
    idx = all_together_fix_2.material == mat
    all_together_fix_2.loc[idx,'checker'] = 0
    
# Keep those with checker = 1
all_together_fix_3 = all_together_fix_2.loc[all_together_fix_2.checker == 1]
# Delete column "checker"
all_together_fix_3 = all_together_fix_3[columns_tokeep]

# Show
print(all_together_fix_3.shape)
all_together_fix_3.head(2)

(38686, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,4,0.084465,-79.050839,627.300049,9 de enero.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,-33.629462,-71.549508,104.600000,9 de enero.csv


## Drop duplicates

In [83]:
# Drop duplicates for item data (itemname, tipo, material, quantity) and location (latitude, longitude)
garbage = all_together_fix_3.drop_duplicates(subset=['itemname','tipo','material','quantity','latitude','longitude'])

# Show
print(garbage.shape)
garbage.head(2)

(17226, 8)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin
0,Latas de aluminio o estaño,NaN,Aluminio,4,0.084465,-79.050839,627.300049,9 de enero.csv
1,Latas de aluminio o estaño,NaN,Aluminio,1,-33.629462,-71.549508,104.600000,9 de enero.csv


## Save to local

In [84]:
garbage.to_csv("../../data/external/temporal_fromjupyter/residuos_arroyovivo.csv",index=False)